# Single Channel Source Separation
Some ideas:
- Deeper networks
- Wider networks
- Different conv filter sizes
- Noisy inputs
- Dropout

## Initial setup

In [9]:
import tensorflow as tf
import numpy as np
import os, errno
import re

In [3]:
def make_directory(f):
    """Makes directory if does not already exist"""
    try:
        os.makedirs(f)
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise

## Layer definitions

In [4]:
def _check_list(arg):
    if isinstance(arg, list):
        try:
            return arg[0], arg[1:]
        except IndexError:
            return arg[0], []
    else:
        return arg, []

def _get_variable_initializer(init_type, var_shape, *args):
    if init_type == "random_normal":
        mean = float(args[0])
        stddev = float(args[1])
        return tf.random_normal(var_shape, mean=mean, stddev=stddev)
    elif init_type == "truncated_normal":
        mean = float(args[0])
        stddev = float(args[1])
        return tf.truncated_normal(var_shape, mean=mean, stddev=stddev)
    elif init_type == "constant":
        c = args[0]
        return tf.constant(c, dtype=tf.float32, shape=var_shape)
    elif init_type == "xavier":
        n_in = tf.cast(args[0], tf.float32)
        return tf.div(tf.random_normal(var_shape), tf.sqrt(n_in))
    else:
        raise ValueError("Variable initializer \"" + init_type + "\" not supported.")

def _apply_normalization(norm_type, x, *args, **kwargs):
    if norm_type == "batch_norm":
        return batch_norm(x, *args, **kwargs)
    else:
        raise ValueError("Normalization type \"" + norm_type + "\" not supported.")

def _apply_activation(activation_type, x, *args):
    if activation_type.lower() == "relu":
        return tf.nn.relu(x, name="Relu")
    elif activation_type.lower() == "leaky_relu":
        return tf.maximum(x, 0.1 * x, name="Leaky_Relu")
    elif activation_type.lower() == "softmax":
        return tf.nn.softmax(x)
    elif activation_type.lower() == "none":
        return x
    else:
        raise ValueError("Activation type \"" + activation_type + "\" not supported.")
        
def conv2d(input_layer,
           num_outputs,
           kernel_size,
           stride=1,
           padding="VALID",
           data_format="NCHW",
           normalizer_fn=None,
           activation_fn=None,
           weights_initializer="random_normal",
           biases_initializer=None,
           trainable=True,
           scope="CONV"):
    with tf.name_scope(scope):
        input_shape = input_layer.get_shape().as_list()
        
        # Create weights
        W_init_type, W_init_params = _check_list(weights_initializer)
        with tf.name_scope(W_init_type + "_initializer"):
            if data_format == "NHWC":
                input_channels = input_shape[3]
            elif data_format == "NCHW":
                input_channels = input_shape[1]
            W_shape = kernel_size + [input_channels, num_outputs]
            if W_init_type == "xavier":
                layer_shape = input_shape[1:]
                n_in = tf.reduce_prod(layer_shape)
                W_init_params = [n_in] 
            W_init = _get_variable_initializer(W_init_type,
                                                W_shape,
                                                *W_init_params)
        W = tf.Variable(W_init, 
                        dtype=tf.float32, 
                        trainable=trainable, 
                        name="weights")
        

        # Convolute input
        stride_h, stride_w = _check_list(stride)
        if isinstance(stride_w, list):
            if len(stride_w) == 0:
                stride_w = stride_h
            else:
                stride_w = stride_w[0]
        if data_format == "NHWC":
            strides = [1, stride_h, stride_w, 1]
        elif data_format == "NCHW":
            strides = [1, 1, stride_h, stride_w]
        out = tf.nn.conv2d(input_layer, 
                            filter=W,
                            strides=strides,
                            padding=padding,
                            data_format=data_format,
                            name="convolution")
        
        # Apply normalization
        if normalizer_fn is not None:
            norm_type, norm_params = _check_list(normalizer_fn)
            out = _apply_normalization(norm_type, 
                                       out, 
                                       *norm_params,
                                       data_format=data_format)
        
        # Add biases
        elif biases_initializer is not None:
            b_init_type, b_init_params = _check_list(biases_initializer)
            if data_format == "NHWC":
                b_shape = [1, 1, 1, num_outputs]
            elif data_format == "NCHW":
                b_shape = [1, num_outputs, 1, 1]
            b_init = _get_variable_initializer(b_init_type,
                                               b_shape,
                                               *b_init_params)
            b = tf.Variable(b_init,
                            dtype=tf.float32,
                            trainable=trainable,
                            name="biases")
            out = tf.add(out, b, name="BiasAdd")

        # Apply activation
        if activation_fn is not None:
            act_type, act_params = _check_list(activation_fn)
            out = _apply_activation(act_type, out, *act_params)

        return out

def conv2d_transpose(x,
                     output_shape,
                     kernel_size,
                     stride=1,
                     padding="VALID",
                     data_format="NCHW",
                     normalizer_fn=None,
                     activation_fn=None,
                     weights_initializer="random_normal",
                     biases_initializer=None,
                     trainable=True,
                     scope="CONV_T"):
    with tf.name_scope(scope):
        x_shape = x.get_shape().as_list()
        
        # Create weights
        W_init_type, W_init_params = _check_list(weights_initializer)
        with tf.name_scope(W_init_type + "_initializer"):
            if data_format == "NHWC":
                input_channels = x_shape[3]
                num_outputs = output_shape[3]
            elif data_format == "NCHW":
                input_channels = x_shape[1]
                num_outputs = output_shape[1]
            W_shape = kernel_size + [num_outputs, input_channels]
            if W_init_type == "xavier": # based on output size
                layer_shape = output_shape[1:]
                n_out = tf.reduce_prod(layer_shape)
                W_init_params = [n_out]
            W_init = _get_variable_initializer(W_init_type,
                                               W_shape,
                                               *W_init_params)
        W = tf.Variable(W_init, 
                        dtype=tf.float32, 
                        trainable=trainable, 
                        name="weights")
        

        # Convolute input
        stride_h, stride_w = _check_list(stride)
        if isinstance(stride_w, list):
            if len(stride_w) == 0:
                stride_w = stride_h
            else:
                stride_w = stride_w[0]
        if data_format == "NHWC":
            strides = [1, stride_h, stride_w, 1]
        elif data_format == "NCHW":
            strides = [1, 1, stride_h, stride_w]
        out = tf.nn.conv2d_transpose(x, 
                                     filter=W,
                                     output_shape=output_shape,
                                     strides=strides,
                                     padding=padding,
                                     data_format=data_format,
                                     name="convolution_transpose")
        
        # Apply normalization
        if normalizer_fn is not None:
            norm_type, norm_params = _check_list(normalizer_fn)
            out = _apply_normalization(norm_type, 
                                       out, 
                                       *norm_params,
                                       data_format=data_format)
        
        # Add biases
        elif biases_initializer is not None:
            b_init_type, b_init_params = _check_list(biases_initializer)
            if data_format == "NHWC":
                b_shape = [1, 1, 1, num_outputs]
            elif data_format == "NCHW":
                b_shape = [1, num_outputs, 1, 1]
            b_init = _get_variable_initializer(b_init_type,
                                               b_shape,
                                               *b_init_params)
            b = tf.Variable(b_init,
                            dtype=tf.float32,
                            trainable=trainable,
                            name="biases")
            out = tf.add(out, b, name="BiasAdd")

        # Apply activation
        if activation_fn is not None:
            act_type, act_params = _check_list(activation_fn)
            out = _apply_activation(act_type, out, *act_params)

        return out
    
def flatten(input_layer, 
            data_format="NCHW",
            scope="FLAT"):
    with tf.name_scope(scope):
        # Grab runtime values to determine number of elements
        input_shape = tf.shape(input_layer)
        input_ndims = input_layer.get_shape().ndims
        batch_size = tf.slice(input_shape, [0], [1])
        layer_shape = tf.slice(input_shape, [1], [input_ndims-1])
        num_neurons = tf.expand_dims(tf.reduce_prod(layer_shape), 0)
        flattened_shape = tf.concat([batch_size, num_neurons], 0)
        if data_format == "NHWC":
            input_layer = tf.transpose(input_layer, perm=[0, 3, 1, 2])
        flat = tf.reshape(input_layer, flattened_shape)
        
        # Attempt to set values during graph building
        input_shape = input_layer.get_shape().as_list()
        batch_size, layer_shape = input_shape[0], input_shape[1:]
        if all(layer_shape): # None not present
            num_neurons = 1
            for dim in layer_shape:
                num_neurons *= dim
            flat.set_shape([batch_size, num_neurons])
        else: # None present
            flat.set_shape([batch_size, None])
        return flat

def fully_connected(input_layer,
                    num_outputs,
                    normalizer_fn=None,
                    activation_fn=None,
                    weights_initializer="random_normal",
                    biases_initializer=None,
                    trainable=True,
                    scope="FC"):
    with tf.name_scope(scope):
        input_shape = input_layer.get_shape().as_list()
        
        # Create weights
        W_init_type, W_init_params = _check_list(weights_initializer)
        with tf.name_scope(W_init_type + "_initializer"):
            W_shape = [input_shape[1], num_outputs]
            if W_init_type == "xavier":
                layer_shape = input_shape[1]
                n_in = tf.reduce_prod(layer_shape)
                W_init_params = [n_in]
            W_init = _get_variable_initializer(W_init_type,
                                            W_shape,
                                            *W_init_params)
        W = tf.Variable(W_init,
                        dtype=tf.float32, 
                        trainable=trainable, 
                        name="weights")
        
        # Multiply inputs by weights
        out = tf.matmul(input_layer, W)

        # Apply normalization
        if normalizer_fn is not None:
            norm_type, norm_params = _check_list(normalizer_fn)
            out = _apply_normalization(norm_type, 
                                       out, 
                                       *norm_params,
                                       data_format=None)

        # Add biases
        elif biases_initializer is not None:
            b_init_type, b_init_params = _check_list(biases_initializer)
            b_shape = [num_outputs]
            b_init = _get_variable_initializer(b_init_type,
                                               b_shape,
                                               *b_init_params)
            b = tf.Variable(b_init,
                            dtype=tf.float32,
                            trainable=trainable,
                            name="biases")
            out = tf.add(out, b, name="BiasAdd")
       
        # Apply activation
        if activation_fn is not None:
            act_type, act_params = _check_list(activation_fn)
            out = _apply_activation(act_type, out, *act_params)

        return out

## Graph building

In [52]:
tf.reset_default_graph()

# Setttings
#data_format = "NHWC" # if using cpu
data_format = "NCHW" # if using gpu
results_dir = "./results/trial_1/"
make_directory(results_dir)
time_context = 30
feat_size = 513
num_sources = 4
eps = 1e-18 # numerical stability
alpha = 0.001 # learning rate

# Data formatting
if data_format == "NHWC":
    input_shape = [None, time_context, feat_size, 1]
    target_shape = [None, time_context, feat_size, num_sources]
    channel_dim = 3
elif data_format == "NCHW":
    input_shape = [None, 1, time_context, feat_size]
    target_shape = [None, num_sources, time_context, feat_size]
    channel_dim = 1
else:
    raise ValueError("Unknown data format \"" + data_format + "\"")
spectrogram = tf.placeholder(tf.float32, 
                             shape=input_shape, 
                             name="magnitude_spectrogram")

# Convolutional layer 1
conv1 = conv2d(spectrogram,
               num_outputs=30,
               kernel_size=[1, 30],
               stride=[1, 4],
               padding="VALID",
               data_format=data_format,
               weights_initializer="xavier",
               biases_initializer=["constant", 0.0],
               scope="CONV_1")

# Convolutional layer 2
conv2 = conv2d(conv1,
               num_outputs=30,
               kernel_size=[int(2*time_context/3), 1],
               stride=[1, 1],
               padding="VALID",
               data_format=data_format,
               weights_initializer="xavier",
               biases_initializer=["constant", 0.0],
               scope="CONV_2")
conv2_flat = flatten(conv2,
                     data_format=data_format,
                     scope="CONV_2_FLAT")

# Fully-connected layer 1 (encoding)
fc1 = fully_connected(conv2_flat,
                      num_outputs=256,
                      activation_fn="relu",
                      weights_initializer="xavier",
                      biases_initializer=["constant", 0.0],
                      scope="FC_1")

batch_size = tf.shape(spectrogram)[0]
conv1_shape = conv1.get_shape().as_list()
conv2_shape = conv2.get_shape().as_list()
conv2_size = conv2_shape[1] * conv2_shape[2] * conv2_shape[3]
fc2, convt1, convt2 = [], [], []
for i in range(num_sources):
    # Fully-connected layer 2 (decoding)
    fc2_i = fully_connected(fc1,
                            num_outputs=conv2_size,
                            activation_fn="relu",
                            weights_initializer="xavier",
                            biases_initializer=["constant", 0.0],
                            scope="FC_2_%d" % (i+1))
    fc2.append(fc2_i)
    
    # Convolutional transpose layer 1
    # Side note: tf.reshape() can infer size of one dimension given rest, so -1 okay
    #            tf.nn.conv2d_transpose() must know exact dimensions, but batch size can
    #                be inferred at runtime using tf.shape()
    fc2_i = tf.reshape(fc2_i, [-1] + conv2_shape[1:])
    convt1_i = conv2d_transpose(fc2_i,
                                output_shape=[batch_size] + conv1_shape[1:],
                                kernel_size=[int(2*time_context/3), 1],
                                stride=[1, 1],
                                padding="VALID",
                                data_format=data_format,
                                weights_initializer="xavier",
                                biases_initializer=["constant", 0.0],
                                scope="CONVT_1_%d" % (i+1))
    convt1.append(convt1_i)
    
    # Convolutional transpose layer 2
    convt2_i = conv2d_transpose(convt1_i,
                                output_shape=[batch_size] + input_shape[1:],
                                kernel_size=[1, 30],
                                stride=[1, 4],
                                padding="VALID",
                                data_format=data_format,
                                weights_initializer="xavier",
                                biases_initializer=["constant", 0.0],
                                scope="CONVT_2_%d" % (i+1))
    convt2.append(convt2_i)

# Output layer
with tf.name_scope("y_hat"):
    convt2_all = tf.concat(convt2, axis=channel_dim)
    b = tf.Variable(tf.constant(0.0, shape=[1, 1, 1, 1]),
                    dtype=tf.float32,
                    name="bias")
    y_hat = tf.maximum(tf.add(convt2_all, b), 0, name="y_hat")

# Masks: m_n(f) = |y_hat_n(f)| / Σ(|y_hat_n'(f)|)
with tf.name_scope("masks"):
    rand = tf.random_uniform([batch_size] + input_shape[1:])
    den = tf.reduce_sum(y_hat, axis=channel_dim, keep_dims=True) + (eps * rand)
    masks = tf.div(y_hat, den, name="masks") # broadcast along channel dimension
    
# Source signals: y_tilde_n(f) = m_n(f) * x(f), 
# where x(f) is the spectrogram of the input mixture signal
with tf.name_scope("y_tilde"):
    y_tilde = tf.multiply(masks, spectrogram, name="y_tilde") # broadcast along channel dimension

# Loss function: L = Σ(||y_tilde_n - target_n||^2)
with tf.name_scope("loss"):
    targets = tf.placeholder(tf.float32, 
                             shape=target_shape, 
                             name="target_sources")
    reduc_indices = [i for i in range(4) if i != channel_dim]
    loss_n = tf.reduce_sum(tf.square(y_tilde - targets), axis=reduc_indices, name="loss_n")
    loss_total = tf.reduce_sum(loss_n, name="loss_total")

# Optimizer
with tf.name_scope("train_step"):
    optimizer = tf.train.AdamOptimizer(alpha)
    train_step = optimizer.minimize(loss_total)

# Summaries
saver = tf.train.Saver(max_to_keep=5)        
graph = tf.get_default_graph()
writer = tf.summary.FileWriter(results_dir, graph)
loss_sum = []
with tf.name_scope("summaries"):
    for i in range(num_sources):
        loss_sum.append(tf.summary.scalar("loss_%d" % (i+1), loss_n[i]))
    loss_sum.append(tf.summary.scalar("loss_total", loss_total))
    loss_sum = tf.summary.merge(loss_sum)


## Training

In [53]:
def get_shape(shape_file):
    """
    Reads a .shape file
    """
    with open(shape_file, 'rb') as f:
        line=f.readline().decode('ascii')
        if line.startswith('#'):
            shape=tuple(map(int, re.findall(r'(\d+)', line)))
            return shape
        else:
            raise IOError('Failed to find shape in file')

(5, 3494, 2049)


In [54]:
def create_batches(data, batch_size):
    batches = []
    time_batches = data.shape[1] // time_context
    freq_batches = data.shape[2] // feat_size
    for t in range(time_batches):
        for f in range(freq_batches):
            batches.append(data[:, t*time_context:(t+1)*time_context, f*feat_size:(f+1)*feat_size])
    return np.asarray(batches)

In [55]:
params_dir = results_dir + "params/"
make_directory(params_dir)

num_epochs = 100
batch_size = 32
f_in = np.fromfile("./features/02-AchLiebenChristen__m_.data")
f_shape = get_shape("./features/02-AchLiebenChristen__m_.shape")
f_in = np.reshape(f_in, f_shape)
input_data = create_batches(f_in[0:1], batch_size)
target_data = create_batches(f_in[1:], batch_size)
iter_size = len(input_data) // batch_size

# Initialize graph
sess = tf.Session()
sess.run(tf.global_variables_initializer())

global_step = 0
for epoch in range(num_epochs):
    train_loss_total = 0
    train_loss_n = 0
    
    for i in range(iter_size):
        # Get batch from input data and target data
        input_batch = input_data[i*batch_size:(i+1)*batch_size] # magnitude spectrogram of whole
        target_batch = target_data[i*batch_size:(i+1)*batch_size] # magnitude spectrogram of sources
        
        # Perform training step
        feed_dict = {spectrogram: input_batch, targets: target_batch}
        loss_sum_, _ = sess.run([loss_sum, train_step], 
                                feed_dict=feed_dict)
        writer.add_summary(loss_sum_, global_step=global_step)
        writer.flush()
        global_step += 1
        
    saver.save(sess, params_dir + "model", 
               global_step=epoch)
    

## Testing

In [49]:
input_data.shape

(348, 1, 30, 513)